In [1]:
import langchain
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.chains import RetrievalQA
from langchain.document_loaders import ArxivLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings, LlamaCppEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate

In [2]:
# Globals
embedding_model = 'hkunlp/instructor-xl'
instruction_model_path = "/Users/bsantanna/dev/workspace/community/Llama-2-13b-chat-hf/ggml-model-f16.bin"
n_gpu_layers = 1
n_batch = 512
n_ctx = 4096

In [3]:
# initialize embeddings model
# embedding = HuggingFaceInstructEmbeddings(model_name=embedding_model)
embedding = LlamaCppEmbeddings(model_path=instruction_model_path,
                               n_gpu_layers=n_gpu_layers,
                               n_batch=n_batch,
                               f16_kv=True)

llama.cpp: loading model from /Users/bsantanna/dev/workspace/community/Llama-2-13b-chat-hf/ggml-model-f16.bin
llama_model_load_internal: format     = ggjt v1 (pre #1405)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 6912
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_head_kv  = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 1.0e-06
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 1 (mostly F16)
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.11 MB
llama_model_load_internal: mem required  = 25192.69 MB (+  400.00 MB per state)
llama_new_context_

In [4]:
# Load LLM instruction following model
llm = LlamaCpp(
    model_path=instruction_model_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=n_ctx,
    f16_kv=True,
    verbose=False,
)

llama.cpp: loading model from /Users/bsantanna/dev/workspace/community/Llama-2-13b-chat-hf/ggml-model-f16.bin
llama_model_load_internal: format     = ggjt v1 (pre #1405)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 4096
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 6912
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_head_kv  = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 1.0e-06
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 1 (mostly F16)
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.11 MB
llama_model_load_internal: mem required  = 25192.69 MB (+ 3200.00 MB per state)
llama_new_context

In [5]:
# Initialize documents
loader = ArxivLoader(query="dynamic scaling in self-similar systems", load_max_docs=20)
documents = loader.load()

In [6]:
# Initialize index
index = VectorstoreIndexCreator(embedding=embedding).from_documents(documents)


llama_print_timings:        load time = 56147.51 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time = 56146.79 ms /   309 tokens (  181.70 ms per token,     5.50 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time = 56148.08 ms

llama_print_timings:        load time = 56147.51 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time = 52896.68 ms /   246 tokens (  215.03 ms per token,     4.65 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time = 52898.08 ms

llama_print_timings:        load time = 56147.51 ms
llama_print_timings:   

In [7]:
# initialize retriever
retriever = index.vectorstore.as_retriever()

In [8]:
# initialize retrieval qa chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=retriever,
    verbose=True
)

In [9]:
# tools = load_tools(['llm-math'], llm=llm)
tools = [
    Tool(
        name="dynamic scaling in self-similar systems QA system",
        func=qa_chain.run,
        description="Useful for when you need to clarify your understanding in problem solving concepts related to dynamic scaling in self-similar systems. Input should be a fully formed question that improves your understanding. Ensure that the question does not make reference to any unclear pronouns used in the preceding conversation.",
    ),
]

In [10]:
# initialize agent
react = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, return_intermediate_steps=True, verbose=True)

In [11]:
# Initialize prompt 
prompt = PromptTemplate.from_template("""
Follow these steps to effectively solve the multiple-choice problem enclosed within <>, considering answers that may vary in precision:

1. Learn from given context until you reach a comprehensive understanding of each concept in the question and can solve the problem with confidence.
   
2. Break down the question to identify incremental steps that lead to the solution. Provide detailed reasoning throughout the process, including analysis of relevant concepts and cross-validation from qa tools to ensure accuracy and confidence in your response.
   
3. Progressively work through the solution, focusing on increasing your confidence in the accuracy of options (A-E) with further validations using qa tools.

4. Assess and eliminate options (A-E) that are incorrect or inaccurate, remember to perform incremental steps including analysis of relevant concepts and cross-validation from qa tools to ensure you are not eliminating the correct option.

5. Validate the remaining options using a cause-and-effect approach along with necessary calculations, performing incremental steps including analysis of relevant concepts and cross-validation from qa tools to ensure accuracy and confidence in your response.

6. Identify the correct option and return the corresponding letter (A-E). Before finalizing your answer, ensure your calculations substantiate its correctness performing incremental steps including analysis of relevant concepts and cross-validation from qa tools to ensure accuracy and confidence in your response.

7. Choose the option from (A-E) that most closely aligns with your conclusion based on the evidence and reasoning you have provided.

<
{question}

{options}
>""")

In [12]:
# Format input
question = """Which of the following is an accurate definition of dynamic scaling in self-similar systems?"""
options = """
A - Dynamic scaling refers to the evolution of self-similar systems, where data obtained from snapshots at fixed times exhibits similarity to the respective data taken from snapshots of any earlier or later time. This similarity is tested by a certain time-dependent stochastic variable x.

B - Dynamic scaling refers to the non-evolution of self-similar systems, where data obtained from snapshots at fixed times is similar to the respective data taken from snapshots of any earlier or later time. This similarity is tested by a certain time-dependent stochastic variable x.
	
C - Dynamic scaling refers to the evolution of self-similar systems, where data obtained from snapshots at fixed times is dissimilar to the respective data taken from snapshots of any earlier or later time. This dissimilarity is tested by a certain time-independent stochastic variable y.

D - Dynamic scaling refers to the non-evolution of self-similar systems, where data obtained from snapshots at fixed times is dissimilar to the respective data taken from snapshots of any earlier or later time. This dissimilarity is tested by a certain time-independent stochastic variable y.

E - Dynamic scaling refers to the evolution of self-similar systems, where data obtained from snapshots at fixed times is independent of the respective data taken from snapshots of any earlier or later time. This independence is tested by a certain time-dependent stochastic variable z.
"""
formatted_input = prompt.format_prompt(
    question=question,
    options=options
)

In [15]:
langchain.debug=True
result = react(formatted_input)
langchain.debug=False

[chain/start] [1:RunTypeEnum.chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:RunTypeEnum.chain:AgentExecutor > 2:RunTypeEnum.chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:RunTypeEnum.chain:AgentExecutor > 2:RunTypeEnum.chain:LLMChain > 3:RunTypeEnum.llm:LlamaCpp] Entering LLM run with input:
{
  "prompts": [
    "Answer the following questions as best you can. You have access to the following tools:\n\ndynamic scaling in self-similar systems QA system: Useful for when you need to clarify your understanding in problem solving concepts related to dynamic scaling in self-similar systems. Input should be a fully formed question that improves your understanding. Ensure that the question does not make reference to any unclear pronouns used in the preceding conversation.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one

In [16]:
print(result['output'])

The correct definition of dynamic scaling in self-similar systems is A - Dynamic scaling refers to the evolution of self-similar systems, where data obtained from snapshots at fixed times exhibits similarity to the respective data taken from snapshots of any earlier or later time. This similarity is tested by a certain time-dependent stochastic variable x.
